In [190]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn import metrics


In [191]:
nba_df_orig = pd.read_csv("c:/Users/user/Downloads/nba_2013.csv")

In [192]:
nba_df_orig.head()

,player,pos,age,bref_team_id,g,gs,mp,fg,fga,fg.,...,drb,trb,ast,stl,blk,tov,pf,pts,season,season_end
0,Quincy Acy,SF,23,TOT,63,0,847,66,141,0.468,...,144,216,28,23,26,30,122,171,2013-2014,2013
1,Steven Adams,C,20,OKC,81,20,1197,93,185,0.503,...,190,332,43,40,57,71,203,265,2013-2014,2013
2,Jeff Adrien,PF,27,TOT,53,12,961,143,275,0.520,...,204,306,38,24,36,39,108,362,2013-2014,2013
3,Arron Afflalo,SG,28,ORL,73,73,2552,464,1011,0.459,...,230,262,248,35,3,146,136,1330,2013-2014,2013
4,Alexis Ajinca,C,25,NOP,56,30,951,136,249,0.546,...,183,277,40,23,46,63,187,328,2013-2014,2013


In [193]:
nba_df_orig.shape

(481, 31)

In [194]:
nba_df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   player        481 non-null    object 
 1   pos           481 non-null    object 
 2   age           481 non-null    int64  
 3   bref_team_id  481 non-null    object 
 4   g             481 non-null    int64  
 5   gs            481 non-null    int64  
 6   mp            481 non-null    int64  
 7   fg            481 non-null    int64  
 8   fga           481 non-null    int64  
 9   fg.           479 non-null    float64
 10  x3p           481 non-null    int64  
 11  x3pa          481 non-null    int64  
 12  x3p.          414 non-null    float64
 13  x2p           481 non-null    int64  
 14  x2pa          481 non-null    int64  
 15  x2p.          478 non-null    float64
 16  efg.          479 non-null    float64
 17  ft            481 non-null    int64  
 18  fta           481 non-null    

In [237]:
nba_df_orig.isna().sum() 


player          0
pos             0
age             0
bref_team_id    0
g               0
gs              0
mp              0
fg              0
fga             0
fg.             0
x3p             0
x3pa            0
x3p.            0
x2p             0
x2pa            0
x2p.            0
efg.            0
ft              0
fta             0
ft.             0
orb             0
drb             0
trb             0
ast             0
stl             0
blk             0
tov             0
pf              0
pts             0
season          0
season_end      0
dtype: int64

In [238]:
nba_df_orig.isnull().sum()

player          0
pos             0
age             0
bref_team_id    0
g               0
gs              0
mp              0
fg              0
fga             0
fg.             0
x3p             0
x3pa            0
x3p.            0
x2p             0
x2pa            0
x2p.            0
efg.            0
ft              0
fta             0
ft.             0
orb             0
drb             0
trb             0
ast             0
stl             0
blk             0
tov             0
pf              0
pts             0
season          0
season_end      0
dtype: int64

In [222]:
nba_df_orig['fg.'].fillna(nba_df_orig['fg.'].mean(),inplace = True)
nba_df_orig['x3p.'].fillna(nba_df_orig['x3p.'].mean(),inplace = True)
nba_df_orig['x2p.'].fillna(nba_df_orig['x2p.'].mean(),inplace = True)
nba_df_orig['efg.'].fillna(nba_df_orig['efg.'].mean(),inplace = True)
nba_df_orig['ft.'].fillna(nba_df_orig['ft.'].mean(),inplace = True)


In [223]:
nba_df_orig.isnull().sum()

player          0
pos             0
age             0
bref_team_id    0
g               0
gs              0
mp              0
fg              0
fga             0
fg.             0
x3p             0
x3pa            0
x3p.            0
x2p             0
x2pa            0
x2p.            0
efg.            0
ft              0
fta             0
ft.             0
orb             0
drb             0
trb             0
ast             0
stl             0
blk             0
tov             0
pf              0
pts             0
season          0
season_end      0
dtype: int64

In [224]:
numerical_cols = []
for col in nba_df_orig.columns:
    if nba_df_orig[col].dtype != 'O':
        numerical_cols.append(col)
        

In [225]:
numerical_cols

['age',
 'g',
 'gs',
 'mp',
 'fg',
 'fga',
 'fg.',
 'x3p',
 'x3pa',
 'x3p.',
 'x2p',
 'x2pa',
 'x2p.',
 'efg.',
 'ft',
 'fta',
 'ft.',
 'orb',
 'drb',
 'trb',
 'ast',
 'stl',
 'blk',
 'tov',
 'pf',
 'pts',
 'season_end']

In [226]:
nba_numerical = nba_df_orig[numerical_cols]
nba_numerical.head()


,age,g,gs,mp,fg,fga,fg.,x3p,x3pa,x3p.,...,orb,drb,trb,ast,stl,blk,tov,pf,pts,season_end
0,23,63,0,847,66,141,0.468,4,15,0.266667,...,72,144,216,28,23,26,30,122,171,2013
1,20,81,20,1197,93,185,0.503,0,0,0.285111,...,142,190,332,43,40,57,71,203,265,2013
2,27,53,12,961,143,275,0.520,0,0,0.285111,...,102,204,306,38,24,36,39,108,362,2013
3,28,73,73,2552,464,1011,0.459,128,300,0.426667,...,32,230,262,248,35,3,146,136,1330,2013
4,25,56,30,951,136,249,0.546,0,1,0.000000,...,94,183,277,40,23,46,63,187,328,2013


In [264]:
x = nba_numerical.drop(columns=['pts','season_end'])
y = nba_numerical['pts']

In [265]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [280]:
# check for neighbouor, for which the score is best
for k in range(15):
    k_value = k + 1
    knn = KNeighborsRegressor(n_neighbors = k_value)
    knn.fit(x_train, y_train) 
    reg_score = knn.score(x_test,y_test)
    print( " Regression score  : ", format(round(reg_score * 100,2),'.2f'), "for neighbours =  ", k_value)

 Regression score  :  97.39 for neighbours =   1
 Regression score  :  97.64 for neighbours =   2
 Regression score  :  98.04 for neighbours =   3
 Regression score  :  98.14 for neighbours =   4
 Regression score  :  98.16 for neighbours =   5
 Regression score  :  98.04 for neighbours =   6
 Regression score  :  98.05 for neighbours =   7
 Regression score  :  97.83 for neighbours =   8
 Regression score  :  97.73 for neighbours =   9
 Regression score  :  97.49 for neighbours =   10
 Regression score  :  97.58 for neighbours =   11
 Regression score  :  97.55 for neighbours =   12
 Regression score  :  97.40 for neighbours =   13
 Regression score  :  97.30 for neighbours =   14
 Regression score  :  97.23 for neighbours =   15


In [281]:
# neighbours  to be considered as 8

In [282]:
knn_model = KNeighborsRegressor(n_neighbors = 8)
knn_model.fit(x_train, y_train) 
y_pred = knn_model.predict(x_test)
reg_score = knn_model.score(x_test,y_test)

In [283]:
reg_score

0.9783486477159687

In [284]:
prediction = pd.DataFrame()

In [285]:
prediction['test'] = list(y_test)

In [286]:
prediction['predicted'] = list(y_pred)

In [287]:
prediction.head()

,test,predicted
0,587,587.625
1,89,90.500
2,350,334.875
3,1417,1432.125
4,1071,1023.625
